In [8]:
import pyspark
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [ ]:
'''SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2'''

In [5]:
rdd = df_green\
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount')\
    .rdd

In [6]:
# Lambada是一种内嵌函数，类似def
# 格式：Lambda arguments: expression
rdd.filter(lambda row: True).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 4, 5, 54), PULocationID=129, total_amount=8.51)]

In [11]:
start = datetime(2020,1,1)
start

datetime.datetime(2020, 1, 1, 0, 0)

In [12]:
rdd.filter(lambda row: row.lpep_pickup_datetime >= start).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 4, 5, 54), PULocationID=129, total_amount=8.51)]

In [13]:
rows = rdd.take(10)
row = rows[0]

In [14]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 11, 4, 0)

In [19]:
# Filter out the pickup_datetime > start date (2020-01-01)
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [20]:
# Formart the pickup_datetime
# Create composite key and composite value
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime\
                .replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key,value) # (key, (amount, count))

In [22]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value #left[0]=amount, left[1]=count
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [29]:
from collections import namedtuple

In [30]:
revenuerow = namedtuple('revenuerow', ['hour','zone','revenue','count'])

In [31]:
def unwrap(row):
    return revenuerow(
        hour = row[0][0], 
        zone = row[0][1], 
        revenue = row[1][0], 
        count = row[1][1])

In [43]:
from pyspark.sql.types import StructType, StructField, IntegerType,TimestampType,DoubleType
schema = StructType([
            StructField('hour', TimestampType(), True), 
            StructField('zone', IntegerType(), True), 
            StructField('revenue', DoubleType(), True), 
            StructField('count', IntegerType(), True)])

In [44]:
# Filter and Map process each row of rdd and throw out results.
# Wrong format: .filter(filter_outliers(row)).
# The def function process the row and return .filter(True)
# Correct format: .filter(filter_outliers)
# The def function automatically process each row
df_results = rdd\
    .filter(filter_outliers)\
    .map(prepare_for_grouping)\
    .reduceByKey(calculate_revenue)\
    .map(unwrap)\
    .toDF(schema)

In [46]:
df_results.write.parquet('tmp/green-revenue')

In [48]:
columns = ['VendorID', 'lpep_pickup_datetime','PULocationID',
               'DOLocationID','trip_distance']
duration_rdd = df_green.select(columns).rdd

In [58]:
import pandas as pd


In [80]:
rows = duration_rdd.take(5)
df = pd.DataFrame(rows, columns=columns)
rows

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 4, 5, 54), PULocationID=129, DOLocationID=129, trip_distance=0.81),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 17, 19, 33, 5), PULocationID=75, DOLocationID=42, trip_distance=2.69),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 12, 41), PULocationID=117, DOLocationID=188, trip_distance=13.11),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 21, 25, 44), PULocationID=41, DOLocationID=151, trip_distance=2.13),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 21, 45, 19), PULocationID=129, DOLocationID=260, trip_distance=0.89)]

In [64]:
def model_predict(df):
    y_pred=df.trip_distance * 5
    return y_pred

In [77]:
# itertuples()
# yield
# RDD to Panda DF
# Panda DF to RDD
# iter() and iterable, iterator
# next() and list()
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predict_duration'] = predictions
    
    for row in df.itertuples():
        yield row

In [79]:
df_predicts = duration_rdd.mapPartitions(apply_model_in_batch).toDF().show(10)

+-----+--------+--------------------+------------+------------+-------------+------------------+
|Index|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|  predict_duration|
+-----+--------+--------------------+------------+------------+-------------+------------------+
|    0|       2|                  {}|         129|         129|         0.81| 4.050000000000001|
|    1|       2|                  {}|          75|          42|         2.69|             13.45|
|    2|       2|                  {}|         117|         188|        13.11|             65.55|
|    3|       2|                  {}|          41|         151|         2.13|10.649999999999999|
|    4|       2|                  {}|         129|         260|         0.89|              4.45|
|    5|       2|                  {}|          75|          75|         0.88|               4.4|
|    6|       2|                  {}|          66|         232|         2.25|             11.25|
|    7|       2|              